In [6]:
import pandas
import great_expectations as gx
from great_expectations import expectations as gxe

In [20]:
# Создаем контекст
context = gx.get_context()

In [24]:
# Загружаем датасет
df = pandas.read_csv("./../data/raw/UCI_Credit_Card.csv")

In [25]:
# Создаем датасурс на основе датасета
data_source_name = "df"
data_source = context.data_sources.add_pandas(name=data_source_name)

In [27]:
# Создаем дата_ассет для нашего датасурса
data_asset_name = "df_asset"
data_asset = data_source.add_dataframe_asset(name=data_asset_name)

In [30]:
# Создаем батч для дата_ассета
batch_definition_name = "df_batch_definition"
batch_definition = data_asset.add_batch_definition_whole_dataframe(
    batch_definition_name
)

In [32]:
# Задаем параметры батча
batch_parameters = {"dataframe": df}

In [ ]:
# Получаем датафрейм как батч
batch = batch_definition.get_batch(batch_parameters=batch_parameters)

In [39]:
# Создаем сьют для проверок
suite_name = "df_expectation_suite"
suite = gx.ExpectationSuite(name=suite_name)
suite = context.suites.add(suite)

In [57]:
# Создаем правила для проверки
expectations = [
    gxe.ExpectColumnToExist(column="LIMIT_BAL"),
    gxe.ExpectColumnValuesToNotBeNull(column="LIMIT_BAL"),
    gxe.ExpectColumnValuesToBeBetween(column="AGE", max_value=100, min_value=18),
    gxe.ExpectColumnDistinctValuesToBeInSet(column="default.payment.next.month", value_set=[0,1])
]

In [58]:
for exp in expectations:
    suite.add_expectation(exp)

In [63]:
# Определяем валидирующий объект
definition_name = "df_validation_definition"
validation_definition = gx.ValidationDefinition(
    data=batch_definition, suite=suite, name=definition_name
)

In [66]:
# Добавляем валидацию в контекст
validation_definition = context.validation_definitions.add(validation_definition)

In [70]:
# Производим валидацию
validation_results = validation_definition.run(batch_parameters=batch_parameters)

Calculating Metrics:   0%|          | 0/16 [00:00<?, ?it/s]

In [84]:
if validation_results.success:
    print('Все проверки пройдены')
else: print(validation_results)

Все проверки пройдены
